In [1]:
%%configure -f
{"executorCores": 4, "executorMemory": "47969M", "driverMemory": "25000M"}

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer
import html2text
from smart_open import smart_open
from time import time
from tqdm import tqdm
import concurrent.futures
from gensim.utils import tokenize

import json

from itertools import islice

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

import scrapy
from scrapy.linkextractors import LinkExtractor
import html2text
from urllib.parse import urlparse, ParseResult
from scrapy.linkextractors import IGNORED_EXTENSIONS
from scrapy.selector import Selector

from flair.models import SequenceTagger
from flair.data import Sentence
from langdetect import detect
from segtok.segmenter import split_single


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1565170755681_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ner_tagger = SequenceTagger.load("ner-ontonotes-fast")
pos_tagger = SequenceTagger.load("pos-fast")

In [5]:
def extract_ne(text, max_sentences=50, max_sentence_length=50):
    text = text.replace("\r", " ").replace("\n", " ")
    sentences = [Sentence(sent, use_tokenizer=True) for sent in split_single(text)]
    sentences = list(islice([sent for sent in sentences if len(sent) < max_sentence_length], max_sentences))
    ner_tagger.predict(sentences)
    
    tags = {}
    for sentence in sentences:
        text = sentence.to_plain_string()
        for entity in sentence.to_dict("ner")["entities"]:
            start_pos = entity["start_pos"]
            end_pos = entity["end_pos"]
            tag_type = entity['type'].lower()
            if tag_type not in tags:
                tags[tag_type] = []

            tags[tag_type].append(text[start_pos:end_pos])
    
    return tags

extract_ne_udf = F.udf(extract_ne, T.MapType(T.StringType(), T.ArrayType(T.StringType())))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def is_english(text):
    try:
        lang = detect(text)
    except:
        return False
    return lang == 'en'
is_english_udf = F.udf(is_english, T.BooleanType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sentence1 = '''
This company was started by Leeroy Jenkins in 2007 and uses nlp and expert knowledge to rush in without a plan.

Leeroy Jenkins founded Moar Dots Inc in 2007.

Its mission is to help raiders kill welps.

* * *  
  
 | WP/ | 2015-06-24 21:29 |  \- |  
 | beta/ | 2019-02-25 01:32 |  \- |  
 | v2/ | 2019-02-25 01:32 |  \- |  
 | v3/ | 2019-02-25 01:32 |  \- |  
  
* * *

'''

sentence2 = '''
This company was started by Leeroy Jenkins in 2007 and uses nlp and expert knowledge to rush in without a plan.

Leeroy Jenkins founded Flaming Whelps Inc in 2007.

Its mission is to help raiders kill whelps.

* * *  
  
 | WP/ | 2015-06-24 21:29 |  \- |  
 | beta/ | 2019-02-25 01:32 |  \- |  
 | v2/ | 2019-02-25 01:32 |  \- |  
 | v3/ | 2019-02-25 01:32 |  \- |  
  
* * *

'''

sentence3 = '''
This company was started by Leeroy Jenkins in 2007 and uses nlp and expert knowledge to rush in without a plan.

Leeroy Jenkins founded Onyxia Wipe Animation Inc in 2007.

Its mission is to help raiders kill welps.

* * *  
  
 | WP/ | 2015-06-24 21:29 |  \- |  
 | beta/ | 2019-02-25 01:32 |  \- |  
 | v2/ | 2019-02-25 01:32 |  \- |  
 | v3/ | 2019-02-25 01:32 |  \- |  
  
* * *

'''

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
extract_ne(sentence1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'person': ['Leeroy Jenkins', 'Leeroy Jenkins'], 'date': ['2007', '2007'], 'org': ['Moar Dots Inc']}

In [21]:
english_companies = (spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/crawler/capiq/keywords")
       .filter((F.length("long_description") > 0) |
               (F.length("description") > 0)
              )
      )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
english_companies.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- long_description: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)

In [14]:
english_companies.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

132580

In [23]:
english_companies.write.parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/english_urls_and_content", 
                                mode="overwrite"
                               )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
english_companies = spark.read.load("s3://onai-ml-dev-eu-west-1/web_crawler/data/english_urls_and_content")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
english_companies.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- long_description: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)

In [26]:
def merge_maps(long_ners, ners):
    ret = {}
    for k,v in long_ners.items():
        if k not in ret:
            ret[k] = set()
        ret[k].update(v)
    for k,v in ners.items():
        if k not in ret:
            ret[k] = set()
        ret[k].update(v)
    return {k:list(v) for k,v in ret.items()}

merge_maps_udf = F.udf(merge_maps, T.MapType(T.StringType(), T.ArrayType(T.StringType())))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
english_ner = (english_companies
               .repartition(1000)
               .select("company_id",
                       "company_name", 
                       "website", 
                       "description", 
                       "long_description", 
                       extract_ne_udf("long_description").alias("long_description_ners"),
                       extract_ne_udf("description").alias("description_ners")
                      )
               .select("company_id",
                       "company_name", 
                       "website", 
                       "description", 
                       "long_description",
                       merge_maps_udf("long_description_ners", "description_ners").alias("ners")
                      )
              )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
english_ner.show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------------------------+---------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
english_ner.write.parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/english_ner", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
tst = spark.read.load("s3://onai-ml-dev-eu-west-1/web_crawler/data/english_ner")

Starting Spark application


In [ ]:
tst.printSchema()

In [ ]:
tst.filter(F.size("ners") > 0).select("company_name", "ners").show(20, False)